In [1]:
from dotenv import load_dotenv
from langchain_google_vertexai import ChatVertexAI, HarmBlockThreshold, HarmCategory
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
import os
import requests

load_dotenv()

False

In [2]:
llm = ChatVertexAI(
    model="gemini-1.0-pro",
    convert_system_message_to_human=True,
    temperature=0.8,
    streaming=True,
    safety_settings={
    },
)

# Basic chain

In [3]:
template = """
    Question: {question}
    Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.invoke(question)



> Entering new LLMChain chain...
Prompt after formatting:

    Question: What NFL team won the Super Bowl in the year Justin Beiber was born?
    Answer: Let's think step by step.

> Finished chain.


{'question': 'What NFL team won the Super Bowl in the year Justin Beiber was born?',
 'text': " Sure, let's break it down.\n\n1. Identify the year Justin Bieber was born: Justin Bieber was born on March 1, 1994.\n2. Determine the corresponding Super Bowl year: The Super Bowl is held in February each year, so we need to look at the Super Bowl that occurred in the 1994 NFL season.\n3. Find the Super Bowl winner: The San Francisco 49ers defeated the San Diego Chargers 49-26 in Super Bowl XXIX, which took place on January 29, 1995.\n\nIn summary, the"}

In [4]:
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools

def load_agent() -> AgentExecutor:
    tools = load_tools(
        tool_names=["ddg-search", "wolfram-alpha", "arxiv", "wikipedia"],
        llm=llm
    )

    return initialize_agent(
        tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )

# Action agent

In [ ]:
from typing import Literal
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.chains.base import Chain
from langchain_experimental.plan_and_execute import (load_chat_planner, load_agent_executor, PlanAndExecute)

ReasoningStrategies = Literal["zero-shot-react", "plan-and-solve"]

def load_agent(
        tool_names: list[str],
        strategy: ReasoningStrategies = "zero-shot-react"
) -> Chain:
    llm = llm
    tools = load_tools(
        tool_names=tool_names,
        llm=llm
    )
    if strategy == "plan-and-solve":
        planner = load_chat_planner(llm)
        executor = load_agent_executor(llm, tools, verbose=True)
        return PlanAndExecute(planner=planner, executor=executor, verbose=True)
    return initialize_agent(
        tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )